# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

The URL here has many query parameters

- `q` for the job search
- This is followed by "+20,000" to return results with salaries (or expected salaries >$20,000)
- `l` for a location 
- `start` for what result number to start on

In [408]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [426]:
def get_ids(salary_range_divider = 90000):
    if salary_range_divider <= 20000:
        return(print("Enter a number larger than $20,000."))
        
    job_ids = pd.DataFrame()
    # Set dividing salaries
    divider_strings = ["+$20000-{}".format(salary_range_divider), "+${}".format(salary_range_divider)]
    
    # Perform two searches, starting with the low-salary jobs
    for level, criterion in enumerate(divider_strings):
        
        result_list = []
        
        # Find the number of results
        URL_for_count = "http://www.indeed.com/jobs?q=data+scientist" + criterion + "&l=Boston"
        soup_for_count = BeautifulSoup(urlopen(URL_for_count).read(), 'html.parser')

        results_number = soup_for_count.find("div", attrs = {"id": "searchCount"}).text
        number_of_results = int(results_number.split(sep = ' ')[-1].replace(',', ''))

        # Now loop through the pages
        i = int(number_of_results/100)
        for page_number in range(i + 1):
            URL_for_results = "http://www.indeed.com/jobs?q=data+scientist{}&l=Boston&start={}".format(criterion, str(100 * page_number))
            soup_for_results = BeautifulSoup(urlopen(URL_for_results).read(), 'html.parser')
            results = soup_for_results.find_all('div', attrs={'data-tn-component': 'organicJob'})
            
            # Extract the ID for each job listing, and include the 'salary_over_X' level
            for x in results:
                result_list.append([x.find('h2', attrs={"class": "jobtitle"})['id'], level])
        
        # Add the job ID numbers
        job_ids = job_ids.append(result_list)
        
    job_ids.columns = ['job_id', "salary_over_${}".format(salary_range)]
    job_ids.drop_duplicates(inplace = True)
    
    return(job_ids)

In [427]:
id_dataframe = get_ids(90000)

In [428]:
id_dataframe

job_id  salary_over_$$20000-90000
0   jl_85c4e91c561780aa                          0
1   jl_e632343c455d80f9                          0
2   jl_05654b2739edb3dc                          0
3   jl_1a7766c45b1abbeb                          0
4   jl_1b5a168dfc7b2712                          0
5   jl_54c0d6b8ad9f9e3d                          0
6   jl_b3462acabd0f43c3                          0
7   jl_d5e17d142783f070                          0
8   jl_d8ed0aa611a75e92                          0
9   jl_28643ddfd474331d                          0
10  jl_cc9965049fa248ee                          0
11  jl_e74603b20ffdd818                          0
12  jl_5ae12f4efea272ad                          0
13  jl_e2707ea0ca1ca78a                          0
14  jl_39b6dfd1dadfb833                          0
15  jl_5e1006a9b5b5ba11                          0
16  jl_c8bf61e39611ec97                          0
17  jl_3a8800e8ef8fa047                          0
18  jl_c49e7c5aea2f1cc1                          0
19  jl_8c0b545bd37b15e3                          0
20  jl_9af256cacb274da6                          0
21  jl_8ddc813ab6f21aa3                          0
22  jl_36c933df8d810383                          0
23  jl_9e33b3c759f172d3                          0
24  jl_649d2a93df3fefca                          0
25  jl_feab5ba8fe68447e                          0
26  jl_82fd0f489c1fdce7                          0
27  jl_16daf037db91e5d2                          0
28  jl_ab7aa67f2f87eeab                          0
29  jl_9baa2d80c25423e7                          0
..                  ...                        ...
46  jl_05d122c5a0ad3761                          1
47  jl_cd9cddef657eb0ef                          1
48  jl_826e413f996787e1                          1
49  jl_0355211d34b6c8da                          1
50  jl_f56b905b2d02273b                          1
51  jl_87f40fcb796da2d6                          1
52  jl_ebc15f25e04090ae                          1
53  jl_eb1ada7e963b6f2b                          1
54  jl_149d31c4f6d79b5e                          1
55  jl_a4bbae96f02d4499                          1
56  jl_506ead9021c7fd81                          1
57  jl_31cd06b99a8d89bf                          1
58  jl_068a3979b3d7481e                          1
59  jl_5a24dcb732d85a93                          1
60  jl_6f1969f9bde9a93f                          1
61  jl_46314eac15b370fb                          1
62  jl_fda42ac0efb2b26b                          1
63  jl_688311d0b8457ad7                          1
64  jl_a5e4263995d2c9b4                          1
65  jl_43481848e1116be4                          1
66  jl_b15a3aecc286f785                          1
67  jl_a15d50dc0c1c2035                          1
68  jl_de2022ddd527014b                          1
69  jl_17337709d0c856bb                          1
70  jl_6417b88c1e033352                          1
71  jl_e2f0fd8d785c83a3                          1
72  jl_69fb1078a82813db                          1
73  jl_7baab1e1b0e8e234                          1
74  jl_1510a78a5449aebd                          1
75  jl_72b70793a17036c8                          1

[149 rows x 2 columns]

Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element="jobTitle`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- **Make sure these functions are robust and can handle cases where the data/field may not be available.**
    - Remember to check if a field is empty or `None` for attempting to call methods on it
    - Remember to use `try/except` if you anticipate errors
- **Test** the functions on the results above and simple examples

In [4]:
## YOUR CODE HERE

Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

#### Complete the following code to collect results from multiple cities and starting points. 
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [5]:
YOUR_CITY = ''

In [6]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 100 # Set this to a high-value (5000) to generate more results. 
# Crawling more results, will also take much longer. First test your code on a small number of results and then expand.

results = []

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
    'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', YOUR_CITY]):
    for start in range(0, max_results_per_city, 10):
        # Grab the results from the request (as above)
        # Append to the full set of results
        pass

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [7]:
## YOUR CODE HERE

Lastly, we need to clean up salary data. 

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly, these will not be useful to us for now
1. Some of the entries may be duplicated
1. The salaries are given as text and usually with ranges.

#### Find the entries with annual salary entries, by filtering the entries without salaries or salaries that are not yearly (filter those that refer to hour or week). Also, remove duplicate entries

In [9]:
## YOUR CODE HERE

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [10]:
## YOUR CODE HERE

### Save your results as a CSV

In [14]:
## YOUR CODE HERE

## Predicting salaries using Logistic Regression

#### Load in the the data of scraped salaries

In [12]:
## YOUR CODE HERE

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

We could also perform Linear Regression (or any regression) to predict the salary value here. Instead, we are going to convert this into a _binary_ classification problem, by predicting two classes, HIGH vs LOW salary.

While performing regression may be better, performing classification may help remove some of the noise of the extreme salaries. We don't have to choice the `median` as the splitting point - we could also split on the 75th percentile or any other reasonable breaking point.

In fact, the ideal scenario may be to predict many levels of salaries, 

In [15]:
## YOUR CODE HERE

#### Thought experiment: What is the baseline accuracy for this model?

In [16]:
## YOUR CODE HERE

#### Create a Logistic Regression model to predict High/Low salary using statsmodel. Start by ONLY using the location as a feature. Display the coefficients and write a short summary of what they mean.

In [17]:
## YOUR CODE HERE

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title 
- or whether 'Manager' is in the title. 
- Then build a new Logistic Regression model with these features. Do they add any value? 


In [18]:
## YOUR CODE HERE

#### Rebuild this model with scikit-learn.
- You can either create the dummy features manually or use the `dmatrix` function from `patsy`
- Remember to scale the feature variables as well!


In [19]:
## YOUR CODE HERE

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy, AUC, precision and recall of the model. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.

In [20]:
## YOUR CODE HERE

#### Compare L1 and L2 regularization for this logistic regression model. What effect does this have on the coefficients learned?

In [21]:
## YOUR CODE HERE

In [22]:
## YOUR CODE HERE

#### Continue to incorporate other text features from the title or summary that you believe will predict the salary and examine their coefficients

#### Take ~100 scraped entries with salaries. Convert them to use with your model and predict the salary - which entries have the highest predicted salaries?

### BONUS 

#### Bonus: Use Count Vectorizer from scikit-learn to create features from the text summaries. 
- Examine using count or binary features in the model
- Re-evaluate the logistic regression model using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [23]:
## YOUR CODE HERE

In [24]:
## YOUR CODE HERE

#### Re-test L1 and L2 regularization. You can use LogisticRegressionCV to find the optimal reguarlization parameters. 
- Re-test what text features are most valuable.  
- How do L1 and L2 change the coefficients?

In [25]:
## YOUR CODE HERE